In [ ]:
import pandas as pd
import json
import os
import glob

In [ ]:
BASE = "/run/user/1000/gvfs/smb-share:server=10.50.1.4,share=datalakev1/02_Salmon/01_Raw_data/01_Detection_frames_cropped/2021/20210721_Huon_AM100_sample(reviewing)"
TARGET_PATH = BASE+"/images/reviewed/batch2(done)"

In [ ]:
def convert_json_to_csv(filename, out):
    s = json.load(open(filename, 'r'))
    
    filename_base = s['imagePath']
    w = s['imageWidth']
    h = s['imageHeight']
    # w = 1360
    # h = 1024

    for ann in s['shapes']:
        label = ann['label']
        xmin = ann['points'][0][0]
        ymin = ann['points'][0][1]
        xmax = ann['points'][1][0]
        ymax = ann['points'][1][1]

        # reversed value check
        if(xmin>xmax): xmin,xmax=swap(xmin,xmax)
        if(ymin>ymax): ymin,ymax=swap(ymin,ymax)

        # check value validity
        if(xmax > w): 
            print(filename, "] out of bound xmax:",xmax)
            xmax = w
            # quit()
        if(ymax > h): 
            print(filename, "] out of bound ymax:",ymax) 
            ymax = h
            # quit()
        if(xmin < 0): 
            print(filename, "] out of bound xmin:",xmin)
            xmin = 0
            # quit()
        if(ymin < 0): 
            print(filename, "] out of bound ymin:",ymin) 
            ymin = 0
            # quit()

        out.write('{},{},{},{},{},{},{},{}\n'.format(
            filename_base, int(w), int(h), label, round(float(xmin),2), round(float(ymin),2), round(float(xmax),2), round(float(ymax),2)))

def swap(x,y):
    temp = x
    x = y
    y = temp
    return x, y

In [ ]:
# define save csv file
out_file = os.path.normpath(TARGET_PATH) + '_hm_reviewed.csv'
out = open(out_file, 'w')
out.write('filename,width,height,class,xmin,ymin,xmax,ymax\n')

json_list = glob.glob(TARGET_PATH+"/*.json")
print(len(json_list), "json files are globbed")

In [ ]:
# convert json to csv
for filename in json_list:
    convert_json_to_csv(filename, out)

out.close()
print("done")